<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/Wikipedia_Events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streaming Wikipedia Events

FROM: https://github.com/donaghhorgan/COMP9033/blob/master/lectures/12%20-%20Streaming%20Data%20Analysis.pdf



In [1]:
#install software
# No clustering for this install !!
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 51.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=9799a9123040b09487d705d545893f0971d29195133b9c063174ff401418253b
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
# start the spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("Wikipedia Event") \
    .getOrCreate()

### Streaming Wikipedia events

Currently, Spark supports three kinds of streaming connection out of the box:

FROM:
https://github.com/donaghhorgan/COMP9033/blob/master/lectures/12%20-%20Streaming%20Data%20Analysis.pdf 

1. Connect to the Wikipedia RecentChanges stream using SSEClient.
2. Create a local socket connection on port 50000.
3. When a client (e.g. Spark) connects to the local socket, relay the next available event to it from the event stream.

In [3]:
!pip install sseclient
from pyspark.streaming import StreamingContext
from sseclient import SSEClient
import threading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5584 sha256=65418c7dc0e294848136b7a87e91844373257fed3ce62e64d693feb7edc3781e
  Stored in directory: /root/.cache/pip/wheels/07/67/7e/96edf627ac746de1a5c5cbb8d59ed960f033b8352dc12c545d
Successfully built sseclient


In [4]:
!rm test.json

msgs = SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')

i=0
with open("test.json", "w") as outfile:
  for e in msgs:
    print(e)
    outfile.write(f"{e}\n")
    i=i+1
    if i>10:
      break

rm: cannot remove 'test.json': No such file or directory

{"$schema":"/mediawiki/recentchange/1.0.0","meta":{"uri":"https://commons.wikimedia.org/wiki/File:Divo_Francisco_Borgi%C3%A6_%C3%A8_Duce_Gandi%C3%A6_tertio,_Soc._Jesu_generali,_%C3%A0_Clemente_X_Pont._Max._sanctorum_clitum_fastis_ad_scripto,_Encomia_emblemate,_prosa,_versu_in_de_salvatoris_Soc._Iesv_ad_S._Clementem,_(14745375491).jpg","request_id":"c4baf7f9-f243-4aa6-8907-c66f3f9286d5","id":"f9becded-5ebe-432b-b081-67fbfd1e9f54","dt":"2022-10-27T12:22:55Z","domain":"commons.wikimedia.org","stream":"mediawiki.recentchange","topic":"eqiad.mediawiki.recentchange","partition":0,"offset":4264435683},"id":2039397674,"type":"edit","namespace":6,"title":"File:Divo Francisco Borgiæ è Duce Gandiæ tertio, Soc. Jesu generali, à Clemente X Pont. Max. sanctorum clitum fastis ad scripto, Encomia emblemate, prosa, versu in de salvatoris Soc. Iesv ad S. Clementem, (14745375491).jpg","comment":"/* wbeditentity-update:0| */ automatically modify [[

In [14]:
# simple thread that read wikipedia upddates and print it in files 
# with 100 update per file

#!rm -rf streaming/*
#!mkdir streaming

import uuid
import json

def read_wikipedia():
    msgs = SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')

    buffer=[]
    i=0
    for e in msgs:
      buffer.append(e)
      if len(buffer)>100:
        with open(f"streaming/{str(uuid.uuid4())}.json", "w") as outfile:
          for m in buffer:
            outfile.write(f"{m}\n")
          outfile.close()
          #json.dump([ob.__dict__ for ob in buffer], outfile)
        buffer.clear()
        i=i+1
        #if i>3:
        #  return

threading.Thread(target=read_wikipedia).start()
#read_wikipedia()


In [5]:
# just extract the schema of a wikipedia update
# required to populate the input table of Structured Stream

import json

#don't forget the r for raw string...
data=r"""{"$schema":"/mediawiki/recentchange/1.0.0","meta":{"uri":"https://www.wikidata.org/wiki/Q114775139","request_id":"31f7d9b1-12c4-4de4-bee7-fbdd8c055c4f","id":"6580fd51-0bde-4f48-8186-8b1f202c4e14","dt":"2022-10-20T13:20:31Z","domain":"www.wikidata.org","stream":"mediawiki.recentchange","topic":"eqiad.mediawiki.recentchange","partition":0,"offset":4248495942},"id":1809138679,"type":"edit","namespace":0,"title":"Q114775139","comment":"/* wbsetclaim-create:2||1 */ [[Property:P6634]]: ellen-rudnick-364a793","timestamp":1666272031,"user":"Korimako","bot":false,"minor":false,"patrolled":true,"length":{"old":2211,"new":2575},"revision":{"old":1754930719,"new":1754930819},"server_url":"https://www.wikidata.org","server_name":"www.wikidata.org","server_script_path":"/w","wiki":"wikidatawiki","parsedcomment":"‎<span dir=\"auto\"><span class=\"autocomment\">Created claim: </span></span> <a href=\"/wiki/Property:P6634\" title=\"Property:P6634\">Property:P6634</a>: ellen-rudnick-364a793"}"""

#don't forget to build a list ie [data]...
rdd=spark.sparkContext.parallelize([data])
df=spark.read.json(rdd)
df.show()

df.printSchema()

#schema is computed ;)
df.schema


+--------------------+-----+--------------------+----------+------------+--------------------+-----+---------+--------------------+---------+--------------------+----------------+------------------+--------------------+----------+----------+----+--------+------------+
|             $schema|  bot|             comment|        id|      length|                meta|minor|namespace|       parsedcomment|patrolled|            revision|     server_name|server_script_path|          server_url| timestamp|     title|type|    user|        wiki|
+--------------------+-----+--------------------+----------+------------+--------------------+-----+---------+--------------------+---------+--------------------+----------------+------------------+--------------------+----------+----------+----+--------+------------+
|/mediawiki/recent...|false|/* wbsetclaim-cre...|1809138679|{2575, 2211}|{www.wikidata.org...|false|        0|‎<span dir="auto"...|     true|{1754930819, 1754...|www.wikidata.org|              

StructType([StructField('$schema', StringType(), True), StructField('bot', BooleanType(), True), StructField('comment', StringType(), True), StructField('id', LongType(), True), StructField('length', StructType([StructField('new', LongType(), True), StructField('old', LongType(), True)]), True), StructField('meta', StructType([StructField('domain', StringType(), True), StructField('dt', StringType(), True), StructField('id', StringType(), True), StructField('offset', LongType(), True), StructField('partition', LongType(), True), StructField('request_id', StringType(), True), StructField('stream', StringType(), True), StructField('topic', StringType(), True), StructField('uri', StringType(), True)]), True), StructField('minor', BooleanType(), True), StructField('namespace', LongType(), True), StructField('parsedcomment', StringType(), True), StructField('patrolled', BooleanType(), True), StructField('revision', StructType([StructField('new', LongType(), True), StructField('old', LongTyp

## Streaming analysis

Now that we have our stream relay set up, we can start to analyse its contents. First, let's initialise a [`SparkContext`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.SparkContext) object, which will represent our connection to the Spark cluster. To do this, we must first specify the URL of the master node to connect to. As we're only running this notebook for demonstration purposes, we can just run the cluster locally, as follows:

Next, we create a [`StreamingContext`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext) object, which represents the streaming functionality of our Spark cluster. When we create the context, we must specify a batch duration time (in seconds), to tell Spark how often it should process data from the stream. Let's process the Wikipedia data in batches of one second:

In [ ]:
# stopping all active queries
# you will understand ;-)
for q in spark.streams.active:
  print("stopping",q.name)
  q.stop()
spark.sql(f"drop table wikipedia")

AnalysisException: ignored

In [7]:
!mkdir streaming
lines = spark \
    .readStream \
    .option("cleanSource","delete") \
    .format("json") \
    .schema(df.schema) \
    .load("streaming") \

results=lines.groupBy("user").count()

query = results \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .option("cleanSource","delete") \
    .queryName("wikipedia") \
    .start()

#query.awaitTermination()
print(query.isActive)

True


In [43]:
spark.sql("select * from wikipedia order by count DESC").show(30)
spark.sql("select count(*) from wikipedia").show()

#!cp test.json streaming

+--------------------+-----+
|                user|count|
+--------------------+-----+
|             AskeBot| 2792|
|      VladimirPF-Bot| 1350|
|              Cewbot| 1296|
|             Liridon| 1254|
|            Renamerr| 1226|
|       SchlurcherBot| 1205|
|             Gbawden| 1100|
|             CennoxX|  906|
|             QBA-bot|  869|
|      BotMultichillT|  796|
|         W like wiki|  752|
|        MalnadachBot|  446|
|        BOT-Twm Crys|  442|
|  InternetArchiveBot|  307|
|AliciaFagervingWM...|  221|
|       Hornstrandir1|  175|
|  Holger Uwe Schmitt|  156|
|    Cryptic-waveform|  139|
|         Dinkun Chen|  132|
|            Luicheto|  129|
|                Coen|  115|
|            جار الله|  111|
|      MubasherKaleem|  103|
|       SSoster (WMB)|  100|
|             DimaBot|   86|
|       Mr.Ibrahembot|   78|
|          Argo Navis|   77|
|      Hotaru Natsumi|   72|
|         Davidbazoki|   70|
|            Gymnicus|   68|
+--------------------+-----+
only showing t

In [38]:
human=lines.filter("bot=false").groupBy("user").count()

humanq = human \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .option("cleanSource","delete") \
    .queryName("human") \
    .start()

#query.awaitTermination()
print(humanq.isActive)

True


In [42]:
spark.sql("select * from human order by count desc").show(30)
spark.sql("select count(*) from human").show()

+-------------------+-----+
|               user|count|
+-------------------+-----+
|            Liridon|  629|
|           Renamerr|  533|
|               YoaR|  505|
|         Infovarius|  470|
|            CennoxX|  465|
|      Hornstrandir1|  142|
|         EN-Jungwon|  128|
|Veronika.kasparkova|  116|
|           Haoreima|  111|
|               Coen|   94|
|     Ooswesthoesbes|   91|
|             Taketa|   80|
|             AndreA|   74|
|            Gbawden|   64|
|            Ipigott|   58|
|  Baokowm 225 Winez|   56|
|           אוריאל כ|   50|
|      SSoster (WMB)|   50|
|          BadzilBot|   46|
|   Simon Villeneuve|   44|
|            XxakixX|   44|
|         Kshetunsky|   43|
|            Moebeus|   42|
|           Maltaper|   42|
|          Rudolfo42|   42|
|           AnRo0002|   40|
|           Nodokara|   39|
|       Huangdan2060|   38|
|              鐵路1|   38|
|    Hey man im josh|   36|
+-------------------+-----+
only showing top 30 rows

+--------+
|count(1)|
+-

In [31]:
vladimir=lines.select("meta").filter("user='VladimirPF-Bot'")

vq = vladimir \
    .writeStream \
    .outputMode("append") \
    .format("memory") \
    .option("cleanSource","delete") \
    .queryName("vlad") \
    .start()

#query.awaitTermination()
print(vq.isActive)

True


In [37]:
vq.stop()
print(vq.isActive)

False


In [35]:
spark.sql("select * from vlad").show(truncate=1024)
spark.sql("select count(*) from vlad").show()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                